In [1]:
import os
import github as gh
import bs4 as bs
import requests
from lxml import etree

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
from tqdm import tqdm

In [2]:
GITHUB_ACCESS_TOKEN=os.getenv('GITHUB_ACCESS_TOKEN')
g=gh.Github(GITHUB_ACCESS_TOKEN)
print(g.get_user())

AuthenticatedUser(login=None)


In [3]:
query='tensorflow language:python created:2021-04-01..2021-04-02'
urls = []
result=g.search_repositories(query)
print(result.totalCount)
print(dir(result))
for repository in result:
    urls.append(repository.html_url)
    

29
['_PaginatedListBase__elements', '_PaginatedListBase__fetchToIndex', '_PaginatedList__contentClass', '_PaginatedList__firstParams', '_PaginatedList__firstUrl', '_PaginatedList__headers', '_PaginatedList__list_item', '_PaginatedList__nextParams', '_PaginatedList__nextUrl', '_PaginatedList__parseLinkHeader', '_PaginatedList__requester', '_PaginatedList__reverse', '_PaginatedList__totalCount', '_Slice', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_couldGrow', '_fetchNextPage', '_getLastPageUrl', '_grow', '_isBiggerThan', '_reversed', 'get_page', 'reversed', 'totalCount']
https://github.com/Z-yq/TensorflowTTS
https://github.com/Kal213/StarGAN-Tutorial-Tensorflow-2.

In [4]:
url='https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras'
url2='https://github.com/bamblebam/image-classification-rps'

In [5]:
options=Options()
options.headless=True
driver=webdriver.Chrome(ChromeDriverManager(path='./').install(),options=options)
driver.get(url2)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_win32.zip
Driver has been saved in cache [./\drivers\chromedriver\win32\91.0.4472.101]


In [6]:
table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")

In [7]:
# links=table.find_elements_by_tag_name('a')
# print(len(links))
# for link in links:
#     href=link.get_attribute('href')
#     if 'tree' in href or '.py' in href:
#         stack.append(href)

# driver.get(url2)
# table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
# links=table.find_elements_by_tag_name('a')
# for link in links:
#     href=link.get_attribute('href')
#     if '/tree/' in href or '.py' in href:
#         stack.append(href)

In [8]:
queue=list()
full_list=list()
links_list = list()

In [9]:
def push_to_queue(links):
    for link in links:
        href=link.get_attribute('href')
        if href in links_list:
            continue
        if '/tree/' in href or '.py' in href:
            links_list.append(href)
            queue.append(href)

In [10]:
def search_through_files(link):
    if '/tree/' in link:
        driver.get(link)
        time.sleep(2.5)
        table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-block']")
        links=table.find_elements_by_tag_name('a')
        push_to_queue(links)
    elif '.py' in link:
        driver.get(link)
        time.sleep(2.5)
        full_list.append(link)
        #preprocessing 

In [11]:
def bfs():
    while queue:
        link=queue.pop(0)
        print(link)
        search_through_files(link)

In [12]:
def get_all_relevant_links(url):
    links_list.append(url)
    driver.get(url)
    table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
    links=table.find_elements_by_tag_name('a')
    for link in links:
        href=link.get_attribute('href')
        if '/tree/' in href or '.py' in href:
            links_list.append(href)
            queue.append(href)
    bfs()
    print("Ended")

In [13]:
res=get_all_relevant_links('https://github.com/Z-yq/TensorflowTTS')

https://github.com/Z-yq/TensorflowTTS/tree/main/.idea
https://github.com/Z-yq/TensorflowTTS/tree/main/configs
https://github.com/Z-yq/TensorflowTTS/tree/main/dataloaders
https://github.com/Z-yq/TensorflowTTS/tree/main/dump
https://github.com/Z-yq/TensorflowTTS/tree/main/models
https://github.com/Z-yq/TensorflowTTS/tree/main/synthesizer
https://github.com/Z-yq/TensorflowTTS/tree/main/trainer
https://github.com/Z-yq/TensorflowTTS/tree/main/utils
https://github.com/Z-yq/TensorflowTTS/blob/main/fastspeech_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/run-test.py
https://github.com/Z-yq/TensorflowTTS/blob/main/tacotron_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_acoustic.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_vocoder.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/fastspeech_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/tacotron_dataloader.py
https://github.com/Z-yq/TensorflowTTS

In [14]:
for i in full_list:
    print(i)

https://github.com/Z-yq/TensorflowTTS/blob/main/fastspeech_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/run-test.py
https://github.com/Z-yq/TensorflowTTS/blob/main/tacotron_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_acoustic.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_vocoder.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/fastspeech_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/tacotron_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/vocoder_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/__init__.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/conformer.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/fastspeech.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/model.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/positional_encoding.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/sw

In [15]:
list_set=set(full_list)
print(len(full_list),len(list_set))

51 51


In [16]:
def getSequentialModel():
    for url in full_list:
        print()
        driver.get(url)
        table=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
        if 'Sequential' in table.text:
            return table.text

In [17]:
print(getSequentialModel())

       context_layer = tf.transpose(context_layer, perm=[0, 2, 1, 3])
        context_layer = tf.reshape(
            context_layer, (batch_size, -1, self.all_head_size)
        )
        outputs = (context_layer, attention_probs) if self.output_attentions else (context_layer,)
        return outputs
class TFFastSpeechSelfOutput(tf.keras.layers.Layer):
    """Fastspeech output of self attention module."""
    def __init__(self, config, **kwargs):
        """Init variables."""
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(
            config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
    def call(self, inputs, training=False):
        """Call logic."""
        hidden_states, input_tensor = inputs
        hidden_states = 